In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import requests
from tools import get_files_from_dir, download_if_new
from corona import calc_effects, vacs, vac_list, vac_list_cum, population

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
current_folder = os.getcwd()
data_folder = os.path.join(current_folder,'corona_data')
try:
    os.mkdir(data_folder)
except:
    print('folder already exists')

folder already exists


### External links

In [5]:
url_vaccinations = 'https://impfdashboard.de/static/data/germany_vaccinations_timeseries_v2.tsv'
url_deliveries = 'https://www.bundesgesundheitsministerium.de/fileadmin/Dateien/3_Downloads/C/Coronavirus/Impfstoff/Lieferprognosen_aller_Hersteller_2._Quartal.pdf'

### Downloading files
(including check if the files are new, if not they are deleted directly)

In [29]:
file_path_vaccinations = download_if_new(url_vaccinations,data_folder)
file_path_deliveries = download_if_new(url_deliveries,data_folder)

The-file germany_vaccinations_timeseries_v2.tsv is not updated
The-file Lieferprognosen_aller_Hersteller_2._Quartal.pdf is not updated


### Preparing csv-data - Vaccinations already done

In [12]:
df = pd.read_csv(file_path_vaccinations,sep='\t')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df.rename(columns={'dosen_biontech_erst_kumulativ':'biontech_1_cum','dosen_biontech_zweit_kumulativ':'biontech_2_cum'
                  ,'dosen_moderna_erst_kumulativ':'moderna_1_cum','dosen_moderna_zweit_kumulativ':'moderna_2_cum'
                  ,'dosen_astrazeneca_erst_kumulativ':'astrazeneca_1_cum','dosen_astrazeneca_zweit_kumulativ':'astrazeneca_2_cum'
                  ,'dosen_johnson_kumulativ':'johnson_1_cum'
                  ,'personen_erst_kumulativ': 'all_1','personen_voll_kumulativ': 'all_full'},inplace=True)
columns_begin = ['date','biontech_1_cum','biontech_2_cum','moderna_1_cum','moderna_2_cum','astrazeneca_1_cum','astrazeneca_2_cum'
                 ,'johnson_1_cum','all_1','all_full']
df['week']= df['date'].apply(lambda x: x.isocalendar()[1])
df['weekday']= df['date'].apply(lambda x: x.isocalendar()[2])

df_total = df[[c for c in columns_begin] + [c for c in df if c not in columns_begin]]
df_total.iloc[-5:]

,date,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,all_1,all_full,dosen_kumulativ,dosen_differenz_zum_vortag,dosen_erst_differenz_zum_vortag,dosen_zweit_differenz_zum_vortag,dosen_biontech_kumulativ,dosen_moderna_kumulativ,dosen_astrazeneca_kumulativ,impf_quote_erst,impf_quote_voll,indikation_alter_dosen,indikation_beruf_dosen,indikation_medizinisch_dosen,indikation_pflegeheim_dosen,indikation_alter_erst,indikation_beruf_erst,indikation_medizinisch_erst,indikation_pflegeheim_erst,indikation_alter_voll,indikation_beruf_voll,indikation_medizinisch_voll,indikation_pflegeheim_voll,dosen_dim_kumulativ,dosen_kbv_kumulativ,week,weekday
152,2021-05-28,23870931,12212002,2793925,1020563,8445686,640836,397807,35508349,14271208,49381750,879533,353735,525798,36082933,3814488,9086522,0.427,0.172,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,34756784,14624966,21,5
153,2021-05-29,23952947,12391127,2831427,1044818,8481074,661979,424275,35689723,14522199,49787647,405897,154906,250991,36344074,3876245,9143053,0.429,0.175,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35088272,14699375,21,6
154,2021-05-30,24014660,12525994,2859344,1064070,8492663,679019,434650,35801317,14703733,50070400,282753,101219,181534,36540654,3923414,9171682,0.430,0.177,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35349207,14721193,21,7
155,2021-05-31,24124786,12769003,2885552,1100325,8531733,706027,476329,36018400,15051684,50593755,523355,175404,347951,36893789,3985877,9237760,0.433,0.181,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,35714793,14878962,22,1
156,2021-06-01,24438661,13166289,2911903,1141026,8586297,737181,559596,36496457,15604092,51540953,947198,394790,552408,37604950,4052929,9323478,0.439,0.188,7644926,5826686,1124640,1761928,4910112,4346229,944820,1001588,2734814,1480457,179820,760340,36071518,15469435,22,2


In [38]:
df = df_total[['date','week','weekday']+vac_list_cum[:-1]+['all_1','all_full']].copy()
df = calc_effects(df,shift = 14)

print('Doses last day:', df['date'].iloc[-1].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-1]/1000000,3))
print('Doses 7 days before:', df['date'].iloc[-8].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-8]/1000000,3))
print('Doses 14 days before:', df['date'].iloc[-15].strftime('%d.%m.%y'),round(df_total['dosen_differenz_zum_vortag'].iloc[-15]/1000000,3))
print('Total doses:', round(df_total['dosen_kumulativ'].iloc[-1]/1000000,3))

print('1st dose: ',round(df['all_1'].iloc[-1]/1000000,3),'mln')
print('Full dose: ',round(df['all_full'].iloc[-1]/1000000,3),'mln')
print('1st dose working: ',round(df['all_1'].iloc[-15]/1000000,3),'mln')
print('Full dose working: ',round(df['all_full'].iloc[-15]/1000000,3),'mln')
print('1st dose last week: ',round((df['all_1'].iloc[-1] - df['all_1'].iloc[-8])/1000000,3),'mln')
print('Full dose last week: ',round((df['all_full'].iloc[-1] - df['all_full'].iloc[-8])/1000000,3),'mln')

print('Doses last week: ',round((df_total['dosen_kumulativ'].iloc[-1] - df_total['dosen_kumulativ'].iloc[-8])/1000000,3),'mln')
#print('Doses last week: ',round((df_total['dosen_kumulativ'].iloc[-1] - df_total['dosen_kumulativ'].iloc[-8])/1000000,3),'mln')
print('Doses 2nd-last week: ',round((df_total['dosen_kumulativ'].iloc[-8] 
                                     - df_total['dosen_kumulativ'].iloc[-15])/1000000,3),'mln')

print()

rel_net_effect = round(df['net_effect'].iloc[-1] / df['all_1'].iloc[-1],3)
print(f'The current net effect per person with current 1st dose is {rel_net_effect}')
rel_net_effect_2nd = round(df['net_effect_2nd'].iloc[-1] / df['all_full'].iloc[-1],3)
print(f'The current net effect per person with current 2nd dose is {rel_net_effect_2nd}')
current_real_net = round(df['rel_net_effect'].iloc[-1],3)
print(f'The current relative net effect on the population is ',current_real_net)
R_to_1 = round(1 / (1-current_real_net),3)
print(f'A R-rate of {R_to_1} is reduced to one due to the vaccination')
real_net_14 = round(df['rel_net_effect_after_14'].iloc[-1],3)
print(f'The relative net effect in 14 days on the population is ',real_net_14)
R_to_1_14 = round(1 / (1-real_net_14),3)
print(f'In 14 days a R-rate of {R_to_1_14} is reduced to 1.0 due to the vaccination')
                                            
df.iloc[-10:,:]

Doses last day: 01.06.21 0.947
Doses 7 days before: 25.05.21 0.649
Doses 14 days before: 18.05.21 0.873
Total doses: 51.541
1st dose:  36.496 mln
Full dose:  15.604 mln
1st dose working:  31.781 mln
Full dose working:  9.99 mln
1st dose last week:  2.445 mln
Full dose last week:  3.183 mln
Doses last week:  5.23 mln
Doses 2nd-last week:  4.614 mln

The current net effect per person with current 1st dose is 0.731
The current net effect per person with current 2nd dose is 0.889
The current relative net effect on the population is  0.321
A R-rate of 1.473 is reduced to one due to the vaccination
The relative net effect in 14 days on the population is  0.373
In 14 days a R-rate of 1.595 is reduced to 1.0 due to the vaccination


,date,week,weekday,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,all_1,all_full,johnson_2_cum,net_effect,net_effect_2nd,rel_net_effect,net_effect_after_14,rel_net_effect_after_14,rel_biontech_14,rel_moderna_14,rel_astrazeneca_14,rel_johnson_14,rel_all_1,rel_all_full,R_to_1,R_to_1_14
147,2021-05-23,20,7,22858402,10342296,2546358,882401,8069653,508415,129781,33604194,11862893,129781,22970300.4700,10651558.3500,0.2764,28340535.6100,0.3410,0.7088,0.0707,0.2175,0.0030,0.4044,0.1428,1.3820,1.5175
148,2021-05-24,21,1,22944137,10492887,2591761,897132,8082301,517078,137152,33755351,12044249,137152,23306961.9300,10822071.3700,0.2805,28480877.1100,0.3427,0.7085,0.0716,0.2168,0.0032,0.4062,0.1449,1.3898,1.5214
149,2021-05-25,21,2,23086254,10795143,2643227,924755,8160984,540448,160516,34050981,12420862,160516,23982713.7300,11159733.4500,0.2886,28756259.8000,0.3460,0.7071,0.0724,0.2168,0.0037,0.4098,0.1495,1.4057,1.5292
150,2021-05-26,21,3,23399142,11361143,2694164,952904,8262457,569948,237559,34593322,13121554,237559,24871248.9400,11764548.4800,0.2993,29258838.9700,0.3521,0.7063,0.0726,0.2158,0.0054,0.4163,0.1579,1.4271,1.5434
151,2021-05-27,21,4,23655403,11835914,2744234,988423,8357170,604579,316494,35073301,13745410,316494,25105121.9500,12278387.2300,0.3021,29700778.8100,0.3574,0.7049,0.0730,0.2151,0.0070,0.4221,0.1654,1.4329,1.5562
152,2021-05-28,21,5,23870931,12212002,2793925,1020563,8445686,640836,397807,35508349,14271208,397807,25556362.6900,12719345.7300,0.3075,30092336.0700,0.3621,0.7032,0.0735,0.2146,0.0087,0.4273,0.1717,1.4441,1.5677
153,2021-05-29,21,6,23952947,12391127,2831427,1044818,8481074,661979,424275,35689723,14522199,424275,25765261.2200,12938026.7200,0.3101,30260822.9300,0.3641,0.7023,0.0741,0.2143,0.0093,0.4295,0.1748,1.4494,1.5727
154,2021-05-30,21,7,24014660,12525994,2859344,1064070,8492663,679019,434650,35801317,14703733,434650,25909524.5800,13097392.2600,0.3118,30369992.7400,0.3655,0.7020,0.0747,0.2139,0.0094,0.4308,0.1769,1.4530,1.5760
155,2021-05-31,22,1,24124786,12769003,2885552,1100325,8531733,706027,476329,36018400,15051684,476329,26214616.3400,13396317.3600,0.3155,30578118.4000,0.3680,0.7012,0.0751,0.2134,0.0103,0.4334,0.1811,1.4608,1.5822
156,2021-06-01,22,2,24438661,13166289,2911903,1141026,8586297,737181,559596,36496457,15604092,559596,26672976.9400,13875978.4000,0.3210,31014058.1300,0.3732,0.7013,0.0749,0.2118,0.0119,0.4392,0.1878,1.4727,1.5954


### 2nd analysis - how much time between the first and the second dose per vaccine

In [9]:
average_for_days = 31

for vac in ['biontech','moderna','astrazeneca']:
    print(vac)
    df1 = df[['date',vac+'_1_cum']].rename(columns={vac+'_1_cum':vac+'_n'})
    df2 = df[['date',vac+'_2_cum']].rename(columns={vac+'_2_cum':vac+'_n', 'date':'date_2nd'})
    vac_max = df2[vac+'_n'].max()
    print('2nd dose:', round(vac_max/1000000,3))
    dfn = pd.merge_asof(df1[(df1[vac+'_n'] <= vac_max)],df2, on=vac+'_n')
    dfn['2nd_days'] = dfn['date_2nd']-dfn['date']
    average_time_to_2nd_dose = (dfn.iloc[-1-average_for_days:-1]['2nd_days'].mean())#.total_days()
    print(f'average_time_to_2nd_dose last {average_for_days} days       ', average_time_to_2nd_dose)
    average_time_to_2nd_dose_last_14 = (dfn.iloc[-15:-1]['2nd_days'].mean())
    print('average_time_to_2nd_dose_last_14     ', average_time_to_2nd_dose_last_14)
    print(dfn.iloc[-1:,:])    
    print()

biontech
2nd dose: 13.166
average_time_to_2nd_dose last 31 days        34 days 19:21:17.419354
average_time_to_2nd_dose_last_14      36 days 17:08:34.285714
          date  biontech_n   date_2nd 2nd_days
119 2021-04-25    13000205 2021-05-31  36 days

moderna
2nd dose: 1.141
average_time_to_2nd_dose last 31 days        36 days 18:34:50.322580
average_time_to_2nd_dose_last_14      36 days 01:42:51.428571
          date  moderna_n   date_2nd 2nd_days
119 2021-04-25    1119572 2021-05-31  36 days

astrazeneca
2nd dose: 0.737
average_time_to_2nd_dose last 31 days        59 days 16:15:29.032258
average_time_to_2nd_dose_last_14      79 days 06:51:25.714285
         date  astrazeneca_n   date_2nd 2nd_days
66 2021-03-03         695243 2021-05-30  88 days



### Read pdf - planned deliveries

In [1]:
from tabula import read_pdf
df_pdf = read_pdf(file_path_deliveries, pages='all')

NameError: name 'file_path_deliveries' is not defined

In [19]:
df_pdf[0].columns = ['NaN','week','biontech','moderna','astra','johnson']
df_deliveries = df_pdf[0][['week','biontech','moderna','astra','johnson']]

In [20]:
#df_deliveries = df_pdf[0][['Woche', 'BioNTech', 'Moderna', 'AstraZeneca','Johnson &']].copy()
#df_deliveries.rename(columns={'Woche':'week', 'BioNTech':'biontech', 'Moderna':'moderna', 'AstraZeneca':'astra','Johnson &':'johnson'},inplace='True')
df_deliveries = df_deliveries[(df_deliveries['week'].str.slice(0,2)=='KW')].fillna(0)
for col in df_deliveries.columns[1:]:
    df_deliveries[col] = np.where(df_deliveries[col] == '-', 0, df_deliveries[col])
    df_deliveries[col] = df_deliveries[col].str.replace('.','').fillna(0)
    df_deliveries[col] = df_deliveries[col].astype(int)
date_end = dt.datetime(2021,1,3)
#df_deliveries['days'] = df_deliveries['week'].str.slice(-2,).astype(int).astype(int)*7
#df_deliveries['week_end'] = df_deliveries['days'].apply(lambda x: date_end + dt.timedelta(days = x))

df_deliveries['week_end'] = df_deliveries['week'].apply(lambda x: date_end + dt.timedelta(days = int(x[-2:])*7))
df_deliveries['week'] = df_deliveries['week'].str.slice(-2).astype(int)
df_deliveries.rename(columns={'astra': 'astrazeneca'},inplace=True)

### Forecast

In [21]:
#based on 2nd-analysis
week_delta = {'biontech':5, 'moderna':5,'astrazeneca':12}

In [22]:
df_week = df[['date','week']+vac_list_cum][(df['weekday'] == 7)].copy()
week_last = df_week['week'].iloc[-1]

for vac in vac_list_cum:
    df_week[vac[:-4]] = df_week[vac]-df_week[vac].shift(1)
df_week = df_week.merge(df_deliveries,how = 'outer', on = 'week')
df_week['astrazeneca'].iloc[23:27] = df_week['astrazeneca'].iloc[23:27].mean()
df_week['date'] = df_week['week'].apply(lambda x: date_end + dt.timedelta(days = x*7))
df_week = df_week[['date','week']+vac_list+vacs+vac_list_cum]
df_week.iloc[-10:-3]

/home/thomas/python/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,week,biontech_1,biontech_2,moderna_1,moderna_2,astrazeneca_1,astrazeneca_2,johnson_1,johnson_2,biontech,moderna,astrazeneca,johnson,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,johnson_2_cum
21,2021-05-23,20,1490407.0,2020422.0,408196.0,181145.0,732377.0,155963.0,74598.0,74598.0,3408210.0,550800.0,465600.0,295150.0,22858402.0,10342296.0,2546358.0,882401.0,8069653.0,508415.0,129781.0,129781.0
22,2021-05-30,21,1156258.0,2183698.0,312986.0,181669.0,423010.0,170604.0,304869.0,304869.0,3409380.0,550800.0,0.0,724300.0,24014660.0,12525994.0,2859344.0,1064070.0,8492663.0,679019.0,434650.0,434650.0
23,2021-06-06,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5135130.0,550800.0,610500.0,513600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2021-06-13,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4585230.0,550800.0,610500.0,240000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2021-06-20,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4035330.0,585600.0,610500.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2021-06-27,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5962320.0,622800.0,610500.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2021-07-04,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5962320.0,622800.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
for i in df_week.index:
    if df_week.iloc[i]['week'] > week_last and df_week.iloc[i]['week'] < 52:
        #print(df_week.iloc[i]['week'])
        for vac in week_delta.keys():
            df_week.at[i,vac + '_2'] = min(df_week.iloc[i-1][vac],df_week.iloc[i-week_delta[vac]][vac + '_1'])
            df_week.at[i,vac + '_1'] = df_week.iloc[i-1][vac]-df_week.at[i,vac + '_2']
            df_week.at[i,'johnson_1'] = df_week.iloc[i-1]['johnson']
            df_week['johnson_2'] = df_week['johnson_1']
        for vac in vac_list:
            df_week.at[i, vac+'_cum'] = df_week.at[i - 1, vac+'_cum'] + df_week.at[i, vac]
df_week = calc_effects(df_week,shift=2)
df_week.iloc[-10:-3]

,date,week,biontech_1,biontech_2,moderna_1,moderna_2,astrazeneca_1,astrazeneca_2,johnson_1,johnson_2,biontech,moderna,astrazeneca,johnson,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,johnson_2_cum,net_effect,net_effect_2nd,rel_net_effect,net_effect_after_14,rel_net_effect_after_14,rel_biontech_14,rel_moderna_14,rel_astrazeneca_14,rel_johnson_14,all_full,all_1,rel_all_1,rel_all_full,R_to_1,R_to_1_14
21,2021-05-23,20,1490407.0000,2020422.0000,408196.0000,181145.0000,732377.0000,155963.0000,74598.0000,74598.0000,3408210.0000,550800.0000,465600.0000,295150.0000,22858402.0000,10342296.0000,2546358.0000,882401.0000,8069653.0000,508415.0000,129781.0000,129781.0000,22970300.4700,10651558.3500,0.2764,28340535.6100,0.3410,0.7088,0.0707,0.2175,0.0030,11862893.0000,33604194.0000,0.4044,0.1428,1.3820,1.5175
22,2021-05-30,21,1156258.0000,2183698.0000,312986.0000,181669.0000,423010.0000,170604.0000,304869.0000,304869.0000,3409380.0000,550800.0000,0.0000,724300.0000,24014660.0000,12525994.0000,2859344.0000,1064070.0000,8492663.0000,679019.0000,434650.0000,434650.0000,25909524.5800,13097392.2600,0.3118,30369992.7400,0.3655,0.7020,0.0747,0.2139,0.0094,14703733.0000,35801317.0000,0.4308,0.1769,1.4530,1.5760
23,2021-06-06,22,46690.0000,3362690.0000,269819.0000,280981.0000,0.0000,0.0000,724300.0000,724300.0000,5135130.0000,550800.0000,610500.0000,513600.0000,24061350.0000,15888684.0000,3129163.0000,1345051.0000,8492663.0000,679019.0000,1158950.0000,1158950.0000,28340535.6100,16846821.6500,0.3410,31582584.7900,0.3801,0.6901,0.0800,0.2057,0.0242,19071704.0000,36842126.0000,0.4433,0.2295,1.5175,1.6130
24,2021-06-13,23,2560424.0000,2574706.0000,171293.0000,379507.0000,362630.0000,247870.0000,513600.0000,513600.0000,4585230.0000,550800.0000,610500.0000,240000.0000,26621774.0000,18463390.0000,3300456.0000,1724558.0000,8855293.0000,926889.0000,1672550.0000,1672550.0000,30369992.7400,20090626.9000,0.3655,34861073.1000,0.4195,0.6950,0.0786,0.1946,0.0317,22787387.0000,40450073.0000,0.4868,0.2742,1.5760,1.7227
25,2021-06-20,24,2154836.0000,2430394.0000,192698.0000,358102.0000,0.0000,610500.0000,240000.0000,240000.0000,4035330.0000,585600.0000,610500.0000,0.0000,28776610.0000,20893784.0000,3493154.0000,2082660.0000,8855293.0000,1537389.0000,1912550.0000,1912550.0000,31582584.7900,23463996.3300,0.3801,37363433.9400,0.4496,0.7042,0.0794,0.1826,0.0338,26426383.0000,43037607.0000,0.5179,0.3180,1.6130,1.8169
26,2021-06-27,25,2544923.0000,1490407.0000,177404.0000,408196.0000,197642.0000,412858.0000,0.0000,0.0000,5962320.0000,622800.0000,610500.0000,0.0000,31321533.0000,22384191.0000,3670558.0000,2490856.0000,9052935.0000,1950247.0000,1912550.0000,1912550.0000,34861073.1000,25730100.0800,0.4195,40045234.9100,0.4819,0.7140,0.0797,0.1747,0.0315,28737844.0000,45957576.0000,0.5530,0.3458,1.7227,1.9301
27,2021-07-04,26,4806062.0000,1156258.0000,309814.0000,312986.0000,0.0000,610500.0000,0.0000,0.0000,5962320.0000,622800.0000,0.0000,0.0000,36127595.0000,23540449.0000,3980372.0000,2803842.0000,9052935.0000,2560747.0000,1912550.0000,1912550.0000,37363433.9400,27803408.5600,0.4496,44468265.5900,0.5351,0.7350,0.0784,0.1582,0.0284,30817588.0000,51073452.0000,0.6146,0.3708,1.8169,2.1511


In [41]:
pd.set_option('display.float_format', '{:.4f}'.format)
df_week[['date','week','rel_net_effect','rel_net_effect_after_14','rel_biontech_14','rel_all_1','rel_all_full','R_to_1','R_to_1_14']].iloc[-15:-3]

,date,week,rel_net_effect,rel_net_effect_after_14,rel_biontech_14,rel_all_1,rel_all_full,R_to_1,R_to_1_14
16,2021-04-18,15,0.1093,0.1681,0.6989,0.1995,0.0669,1.1227,1.2021
17,2021-04-25,16,0.1366,0.1983,0.6912,0.2366,0.0721,1.1581,1.2473
18,2021-05-02,17,0.1681,0.2381,0.7189,0.2845,0.0806,1.2021,1.3124
19,2021-05-09,18,0.1983,0.2764,0.7164,0.3303,0.0953,1.2473,1.3820
20,2021-05-16,19,0.2381,0.3118,0.7180,0.3718,0.1135,1.3124,1.4530
21,2021-05-23,20,0.2764,0.3410,0.7088,0.4044,0.1428,1.3820,1.5175
22,2021-05-30,21,0.3118,0.3655,0.7020,0.4308,0.1769,1.4530,1.5760
23,2021-06-06,22,0.3410,0.3801,0.6901,0.4433,0.2295,1.5175,1.6130
24,2021-06-13,23,0.3655,0.4195,0.6950,0.4868,0.2742,1.5760,1.7227
25,2021-06-20,24,0.3801,0.4496,0.7042,0.5179,0.3180,1.6130,1.8169


In [37]:
df_week_3 = df_week[:-3].copy()
print('date: ',df_week_3.date.iloc[-1])
print('1st dose: ',round(df_week_3['all_1'].iloc[-1]/1000000,3),'mln')
print('Full dose: ',round(df_week_3['all_full'].iloc[-1]/1000000,3),'mln')
print('1st dose working: ',round(df_week_3['all_1'].iloc[-3]/1000000,3),'mln')
print('Full dose working: ',round(df_week_3['all_full'].iloc[-3]/1000000,3),'mln')

print()

rel_net_effect = round(df_week_3['net_effect'].iloc[-1] / df_week_3['all_1'].iloc[-1],3)
print(f'The current net effect per person with current 1st dose is {rel_net_effect}')
rel_net_effect_2nd = round(df_week_3['net_effect_2nd'].iloc[-1] / df_week_3['all_full'].iloc[-1],3)
print(f'The current net effect per person with current 2nd dose is {rel_net_effect_2nd}')
current_real_net = round(df_week_3['rel_net_effect'].iloc[-1],3)
print(f'The current relative net effect on the population is ',current_real_net)
R_to_1 = round(1 / (1-current_real_net),3)
print(f'A R-rate of {R_to_1} is reduced to one due to the vaccination')
real_net_14 = round(df_week_3['rel_net_effect_after_14'].iloc[-1],3)
date_14 = df_week_3.date.iloc[-1] + dt.timedelta(days = 14)
print(f'The relative net effect in 14 days ({date_14}) on the population is ',real_net_14)
R_to_1_14 = round(1 / (1-real_net_14),3)
print(f'In 14 days at the {date_14} a R-rate of {R_to_1_14} is reduced to 1.0 due to the vaccination')
                                            
df.iloc[-10:,:]

date:  2021-07-04 00:00:00
1st dose:  51.073 mln
Full dose:  30.818 mln
1st dose working:  43.038 mln
Full dose working:  26.426 mln

The current net effect per person with current 1st dose is 0.674
The current net effect per person with current 2nd dose is 0.88
The current relative net effect on the population is  0.414
A R-rate of 1.706 is reduced to one due to the vaccination
The relative net effect in 14 days (2021-07-18 00:00:00) on the population is  0.494
In 14 days at the 2021-07-18 00:00:00 a R-rate of 1.976 is reduced to 1.0 due to the vaccination


,date,week,weekday,biontech_1_cum,biontech_2_cum,moderna_1_cum,moderna_2_cum,astrazeneca_1_cum,astrazeneca_2_cum,johnson_1_cum,all_1,all_full,johnson_2_cum,net_effect,net_effect_2nd,rel_net_effect,net_effect_after_14,rel_net_effect_after_14,rel_biontech_14,rel_moderna_14,rel_astrazeneca_14,rel_johnson_14,rel_all_1,rel_all_full
147,2021-05-23,20,7,22858402,10342296,2546358,882401,8069653,508415,129781,33604194,11862893,129781,19836964.9500,10165588.8500,0.2387,24816862.4500,0.2986,0.7489,0.0807,0.1667,0.0037,0.4044,0.1428
148,2021-05-24,21,1,22944137,10492887,2591761,897132,8082301,517078,137152,33755351,12044249,137152,20139532.2500,10336690.5500,0.2424,24963205.8500,0.3004,0.7485,0.0817,0.1660,0.0038,0.4062,0.1449
149,2021-05-25,21,2,23086254,10795143,2643227,924755,8160984,540448,160516,34050981,12420862,160516,20737720.4500,10664514.3500,0.2496,25241554.0000,0.3037,0.7471,0.0825,0.1659,0.0045,0.4098,0.1495
150,2021-05-26,21,3,23399142,11361143,2694164,952904,8262457,569948,237559,34593322,13121554,237559,21521415.5000,11236905.6000,0.2590,25755335.3500,0.3099,0.7462,0.0825,0.1648,0.0065,0.4163,0.1579
151,2021-05-27,21,4,23655403,11835914,2744234,988423,8357170,604579,316494,35073301,13745410,316494,21742647.7500,11705232.6500,0.2616,26206876.7500,0.3154,0.7448,0.0827,0.1641,0.0085,0.4221,0.1654
152,2021-05-28,21,5,23870931,12212002,2793925,1020563,8445686,640836,397807,35508349,14271208,397807,22146837.0500,12120258.4500,0.2665,26603015.5500,0.3201,0.7429,0.0831,0.1636,0.0105,0.4273,0.1717
153,2021-05-29,21,6,23952947,12391127,2831427,1044818,8481074,661979,424275,35689723,14522199,424275,22345947.2000,12330756.6000,0.2689,26777973.3500,0.3222,0.7418,0.0838,0.1633,0.0111,0.4295,0.1748
154,2021-05-30,21,7,24014660,12525994,2859344,1064070,8492663,679019,434650,35801317,14703733,434650,22487848.9000,12483082.2000,0.2706,26895709.1000,0.3237,0.7414,0.0843,0.1629,0.0113,0.4308,0.1769
155,2021-05-31,22,1,24124786,12769003,2885552,1100325,8531733,706027,476329,36018400,15051684,476329,22766140.3000,12768336.6000,0.2740,27115070.8000,0.3263,0.7405,0.0846,0.1625,0.0123,0.4334,0.1811
156,2021-06-01,22,2,24438661,13166289,2911903,1141026,8586297,737181,559596,36496457,15604092,559596,23190037.5000,13234429.9000,0.2791,27554525.4500,0.3316,0.7403,0.0843,0.1612,0.0142,0.4392,0.1878
